## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [30]:
import pandas as pd #загрузка библиотеки pandas

In [31]:
data = pd.read_csv('/datasets/data.csv') #чтение файла
data.head(7) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем


### Шаг 2. Предобработка данных

Представленная выборка для целей анализа по основным параметрам потенциальных заемщиков из 21 525 позиций имеет ряд некорректностей: 'children'- по 47 строкам количество (-1) и преобладают заемщики без детей- 66% , отрицательные, вероятно возникли из-за некорректного заполнения входной формы даных- некоректный формат данных или перенос данных в системах (формат выгрузки необходимо привезти в значения без "-") и  многократнозавышенные положительные значения (скорее скорректировать по median, на текущий момент оставлены без изменения) по 'days_employed', 'dob_years'- 101 строка с возрастом "0" и примерно одинаковые пропорции в выборке по возрастным категориям заемщиков, 'education'- затроение данных- в наименованиях разный регистр и шрифт, 'education_id'- на 71% заемщики со средним образованием, 'family_status'- категория с большой буквы "Не женат / не замужем", 'family_status_id' - выборка на 58% из женат/замужем, 'gender'- есть 1 значение "XNA", 'gender' - 66% заемщиков- это женщины, 'income_type' - сотрудников из них большинство (52%), 'debt'- из них 92%- благонадежные заемщики по предыдущей кредитной истории, 'total_income'- выборка с большим разбросом сумм, 'purpose'- представленная категория требует лематизации.

### Обработка пропусков

In [2]:
data.isna().sum() #проверка выборки на предмет пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Наблюдается потенциальная симметричность в пропусках.

In [3]:
data['days_employed'] = data['days_employed'].fillna(0) #заполнение пропущенных значений

In [4]:
data['total_income'] = data['total_income'].fillna(0)

In [33]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [35]:
max(data['days_employed']) #поиск max значения по столбцу'days_employed'

401755.40047533

In [7]:
data['children'] = data['children'].abs() #замена отрицательных значений на положительные

In [8]:
data['days_employed'] = data['days_employed'].abs()

In [9]:
data['children'].value_counts() #проверка замены отрицательных значений

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [10]:
data['days_employed'].value_counts()

0.000000       2174
1645.463049       1
6620.396473       1
1238.560080       1
3047.519891       1
               ... 
2849.351119       1
5619.328204       1
448.829898        1
1687.038672       1
206.107342        1
Name: days_employed, Length: 19352, dtype: int64

   ### Вывод 
В соответствии с методом value_count() проведена оценка выборки в целом (вывод по данным выше), а метод .isna().sum() показывает потенциальную симметричность в количестве пропущенных значений 2174 по столбцам: 'days_employed' и 'total_income'- причиной появления пропусков, например, могла стать закрытая информация по высоким зарплатам. Для осуществления математических расчетов по выборке в дальнейшнем меняем пропуски на нулевые значения. Методом .fillna() произведена замена пропусков по столбцам указанным ранее. Также осуществлена замена отрицательных значений (значение 20 детей оставлено без правок- в реальной жизни такое количество м.б. у заемщика- гос.поддержка кредитования многодетных семей и т.п.) по столбцам 'children' и 'days_employed' методом .abs(). По столбцу 'days_employed' зафиксирован стаж в 1100 лет!? (или 401 755 дней)- вероятно данные сохранены еще и в минутах. В 'dob_years'- нулевые строки (101 значение) и 'gender'- 1 значение "XNA"- изменения не проводились т.к. их  объем в выборке незначителен.

### Замена типа данных

In [11]:
data['days_employed'] = data['days_employed'].astype('int64') #замена типа данных с float64 на int64

In [12]:
data['total_income'] = data['total_income'].astype('int64')

In [13]:
data.info() #проверка замены на int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод 
Посредством метода .astype заменяем тип данных с float64 на int64 по столбцам 'days_employed' и 'total_income' для проведения с данными столбцами математических действий в дальнейших расчетах по проекту.

### Обработка дубликатов

In [14]:
data['education'].value_counts() #проверка наличия дубликатов

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [15]:
data['education_clear'] = data['education'].str.lower() #замена названий в столбце на нижний регистр

In [16]:
data['education_clear'].unique() #уникальные значения

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Вывод 
Столбец 'education' приведен к идентичным значениям по 5 категориям с помощью метода .str.lower() - появился столбец 'education_clear' как верный (проверен на уникальность методом .unique()).

### Лемматизация

In [17]:
print(data['purpose'].unique()) #уникальные значения

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [18]:
from pymystem3 import Mystem #импорт библиотеки с функцией лемматизации
from collections import Counter #импорт библиотеки с функцией счетчика
m = Mystem()

def lemmatize_data(row): # функция для лемматизации строк
    lemmas = m.lemmatize(row)
    return lemmas
 
data['purpose'] = data['purpose'].apply(lemmatize_data) #применена функция лемматизации к столбцу 'purpose' датафрейма, добавлен столбец 'purpose_lemmas' 

print('Словарь лемм столбца "purpose": ', Counter(data['purpose'].sum())) #просуммировали все значения столбца 'purpose_lemmas' и применили к ним счетчик Counter()

Словарь лемм столбца "purpose":  Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})


### Вывод
Лемматизация представяет собой- процесс приведение слова к его словарной форме. Данный метод необходим в проекте для дальнейшей категоризации данных на основе полученного Counter(data['purpose'] при участии метода .apply().

### Категоризация данных

In [19]:
def lemm(series_row): # функция для категориизации данных, выделены 5 категорий и добавлен столбец 'purpose_top5'
    
    if 'жилье' in series_row:
        return 'жилье'
    elif 'недвижимость' in series_row:
        return 'недвижимость'
    elif 'свадьба' in series_row:
        return 'свадьба'
    elif 'автомобиль' in series_row:
        return 'автомобиль'
    elif 'образование' in series_row:
        return 'образование'
    
data['purpose_top5'] = data['purpose'].apply(lemm)

In [20]:
print(data['purpose_top5'].unique()) #уникальные значения

['жилье' 'автомобиль' 'образование' 'свадьба' 'недвижимость']


In [21]:
data['purpose_top5'].value_counts() #проверка факта замены

недвижимость    6367
жилье           4473
автомобиль      4315
образование     4022
свадьба         2348
Name: purpose_top5, dtype: int64

### Вывод 
Процесс категоризации представляет собой-  объединение данных в категории для оптимального хранения и использования (по возрасту, уровню зарплаты и тп) с созданием файла-словаря (метода .apply()).  Проверка на уникальность категоризированных значений выборки успешно пройдена. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
debt_children = pd.DataFrame() #конструктор датафрейма для создания таблицы

debt_children['sum_children'] = data.groupby('children')['debt'].sum() #группировка данных 
debt_children['count_children'] = data.groupby('children')['debt'].count()

debt_children['result_children'] = debt_children['sum_children'] / debt_children['count_children'] #расчет зависимости событий 
debt_children.sort_values('result_children', ascending = False) #сортировка таблицы по столбцу 'result_children'

,sum_children,count_children,result_children
children,,,
20,8,76,0.105263
4,4,41,0.097561
2,194,2055,0.094404
1,445,4865,0.091470
3,27,330,0.081818
0,1063,14149,0.075129
5,0,9,0.000000


### Вывод 
По бездетным просрочка наименьшая, а при росте количества детей просроченная задолженность становится более вероятна на примере данной выборки. Исключением выведенной последовательности является просрочка с 3 детьми < чем с 1 ребенком, возможно влияние субсидий для погашения или меньшая ставка кредита для многодетных семей. Данные по земщикам с 5 детьми незначительны по количеству в общей выборке и допустимы к исключению- возможная причина некорректный формат данных строк (sum=0). 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
debt_family = pd.DataFrame() #конструктор датафрейма для создания таблицы

debt_family['sum_family_status'] = data.groupby('family_status')['debt'].sum() #группировка данных 
debt_family['count_family_status'] = data.groupby('family_status')['debt'].count()

debt_family['result_family_status'] = debt_family['sum_family_status'] / debt_family['count_family_status'] #расчет зависимости событий
debt_family.sort_values('result_family_status', ascending = False) #сортировка таблицы по столбцу 'result_family_status'

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2813,0.097405
гражданский брак,388,4177,0.092890
женат / замужем,931,12380,0.075202
в разводе,85,1195,0.071130
вдовец / вдова,63,960,0.065625


### Вывод > 
Прослеживается следующая тенденция, чем человек ближе к официальному браку (или прошедшие его) по статусу в предложенной выборке, тем процент невозврата ниже (фактор ответственности за семью работает!)- для банка выдача кредита более надежна для нужд семей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
#min(data['total_income'])
max(data['total_income']) #поиск max значения по столбцу'total_income'

2265604

In [29]:
data['total_income'].median().astype('int64') #mediana по 'total_income'

135514

In [26]:
def total_income_metod(value): # функция для перебора зависимости в total_income
    if value >= 300000:
        return 6
    else:
        value = value // 50000
        return value

    #категоризация заемщиков по доходам для определения зависимости- выделено 5 категорий.    
data['total_income_id'] = data['total_income'].apply(total_income_metod) 
data['total_income_id'].value_counts() #расчет по 'total_income'


debt_total_income = pd.DataFrame() #конструктор датафрейма для создания таблицы

debt_total_income['sum'] = data.groupby('total_income_id')['debt'].sum() #группировка данных 
debt_total_income['count'] = data.groupby('total_income_id')['debt'].count()

debt_total_income['result_total_income'] = debt_total_income['sum'] / debt_total_income['count'] #расчет зависимости событий
debt_total_income.sort_values('result_total_income', ascending = False) #сортировка таблицы по столбцу 'result_total_income'

,sum,count,result_total_income
total_income_id,,,
3,368,4118,0.089364
2,491,5704,0.086080
1,331,4091,0.080909
0,193,2546,0.075805
4,164,2254,0.072760
6,106,1482,0.071525
5,88,1330,0.066165


### Вывод 
Уровень дохода по данной выборке  оказывает обратно пропорциональное влияние на вероятность возврата кредита: больше зарплата- меньше вероятности невозврата кредита. Однако при получившейся медиане в 135 514 руб. ограничением для показательной оценки зависимости взята сумма зарплат до 300 000 руб. и тогда наибольший риск невозврата имеют заемщики с диапазоном зарплат 150 000 - 200 000 руб. (необдуманные, дорогие покупки).

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
debt_purpose_top5 = pd.DataFrame() #конструктор датафрейма для создания таблицы 
debt_purpose_top5['sum_purpose_top5'] = data.groupby('purpose_top5')['debt'].sum() #группировка данных 
debt_purpose_top5['count_purpose_top5'] = data.groupby('purpose_top5')['debt'].count()
debt_purpose_top5['result_purpose_top5'] = debt_purpose_top5['sum_purpose_top5'] / debt_purpose_top5['count_purpose_top5'] #расчет зависимости событий
debt_purpose_top5.sort_values('result_purpose_top5', ascending = False) #сортировка таблицы по столбцу 'result_purpose_top5'

,sum_purpose_top5,count_purpose_top5,result_purpose_top5
purpose_top5,,,
автомобиль,403,4315,0.093395
образование,370,4022,0.091994
свадьба,186,2348,0.079216
недвижимость,474,6367,0.074446
жилье,308,4473,0.068858


### Вывод 
Наиболее рискованные для банка кредиты исходя из вероятности невозврата- небольшие суммы на образование, приобретение автомобиля, а  по недвижимости/ жилью риски невозврата гораздо ниже при значительно более высоких займах.

### Шаг 4. Общий вывод 
Наиболее рискованные  заемщики: без семейного статуса и/или с большим количеством детей, при этом выше доход заемщика- меньше вероятность невозврата кредита. Также небольшие суммы кредита (образование, автомобиль) имеют риски невозврата выше, поэтому в реальной жизни кредитная ставка у них значительно выше.